In [6]:
import math
import heapq
from itertools import combinations
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from math import gcd
from tqdm import tqdm

In [7]:
def frobenius_number_nijenhuis(numbers):
    """
    Вычисляет число Фробениуса по алгоритму Нийенхуиса.
    Работает для множества взаимно простых чисел.
    """
    numbers = sorted(numbers)
    min_num = numbers[0]
    n = min_num

    if min_num == 1:
        return 0

    edges = [[] for _ in range(n)]
    temp_g = [{} for _ in range(n)]

    for i in range(n):
        for num in numbers:
            v = (num + i) % n
            if v not in temp_g[i]:
                temp_g[i][v] = num
            else:
                if temp_g[i][v] > num:
                    temp_g[i][v] = num

    for i in range(n):
        for v, w in temp_g[i].items():
            edges[i].append((v, w))

    INF = float('inf')
    dist = [INF] * n
    dist[0] = 0

    heap = [(0, 0)]
    while heap:
        d_v, v = heapq.heappop(heap)
        if d_v != dist[v]:
            continue
        for u, w in edges[v]:
            if dist[u] > dist[v] + w:
                dist[u] = dist[v] + w
                heapq.heappush(heap, (dist[u], u))

    max_dist = max(dist[1:]) if n > 1 else 0
    result = max_dist - n
    return result


In [8]:
def simulate_path_graph(edge_lengths: list[int]) -> tuple[int, list[int]]:
    points = list()
    stabilization_times = [-1 for _ in range(len(edge_lengths))]
    points.append([0, 1])
    vertex_positions = [0 for _ in range(len(edge_lengths) + 1)]
    for i in range(len(edge_lengths)):
        vertex_positions[i + 1] = vertex_positions[i] + edge_lengths[i]
    t = 1
    while True:
        for point in points:
            point[0] += point[1]
        new_points = list()
        for point in points:
            if point[0] == 0 or point[0] == sum(edge_lengths):
                point[1] *= -1
            elif point[0] in vertex_positions and sum([p[0] == point[0] for p in points]) > 1:
                pass
            elif point[0] in vertex_positions:
                new_points.append([point[0], point[1] * (-1)])

        points = points + new_points
        for i in range(len(edge_lengths)):
            count = 0
            for point in points:
                if vertex_positions[i] < point[0] < vertex_positions[i + 1]:
                    count += 1
                if vertex_positions[i] == point[0] and point[1] == 1:
                    count += 1
                if vertex_positions[i + 1] == point[0] and point[1] == -1:
                    count += 1
            if count == edge_lengths[i]:
                if stabilization_times[i] == -1:
                    stabilization_times[i] = t
        if len(points) == sum(edge_lengths):
            return t, stabilization_times
        t += 1

In [9]:
def generate_coprime_triplets():
    numbers = range(5, 70)
    triplets = []

    for triplet in combinations(numbers, 3):
        a, b, c = triplet
        if math.gcd(math.gcd(a, b), c) == 1:
            triplets.append((a, b, c))
            triplets.append((a, c, b))
            triplets.append((b, a, c))
            triplets.append((b, c, a))
            triplets.append((c, a, b))
            triplets.append((c, b, a))

    return triplets


def generate_features(x, y, z):
    return [x, y, z,
            min(x, y), min(x, z), min(y, z), min(x, y, z),
            max(x, y, z), max(x, z), max(x, y), max(x, z),
            gcd(x, y), gcd(x, z), gcd(y, z),
            x // gcd(x, y), y // gcd(x, y), x // gcd(x, z),
            z // gcd(x, z), y // gcd(y, z), z // gcd(y, z),
            frobenius_number_nijenhuis([x, y, z]),
            frobenius_number_nijenhuis([x, x + y, x + y + z]),
            frobenius_number_nijenhuis([x, x + z, x + y + z]),
            frobenius_number_nijenhuis([y, y + x, x + y + z]),
            frobenius_number_nijenhuis([y, y + z, x + y + z]),
            frobenius_number_nijenhuis([z, z + x, x + y + z]),
            frobenius_number_nijenhuis([z, z + y, x + y + z]),
            frobenius_number_nijenhuis([x // gcd(x, y), y // gcd(x, y)]),
            frobenius_number_nijenhuis([x // gcd(x, z), z // gcd(x, z)]),
            frobenius_number_nijenhuis([y // gcd(y, z), z // gcd(y, z)])]

In [10]:
for index in range(4):
    if 0 <= index <= 2:
        print(f"\n------------ существование полинома 2 степени для t_stab[{index}] ------------\n")
    else:
        print("\n------------- существование полинома 2 степени для t_total --------------\n")
    triplets = generate_coprime_triplets()[:5000]

    A = []
    B = []

    for a, b, c in tqdm(triplets, "Подсчет времени стабилизации"):
        t_val, t_stab = simulate_path_graph([a, b, c])
        stab_times = t_stab + [t_val]
        row = generate_features(a, b, c)

        A.append(row)
        B.append(stab_times[index])

    poly = PolynomialFeatures(degree=2, include_bias=False)
    X_poly = poly.fit_transform(A)

    print(f"Размерность исходных признаков: {len(A[0])}")
    print(f"Размерность полиномиальных признаков: {X_poly.shape[1]}")

    A = np.array(X_poly, dtype=np.float64)
    B = np.array(B, dtype=np.float64)

    rank_A = np.linalg.matrix_rank(A)
    rank_Ab = np.linalg.matrix_rank(np.column_stack((A, B)))

    print(f"Ранг матрицы A: {rank_A}")
    print(f"Ранг расширенной матрицы [A|B]: {rank_Ab}")

    if rank_A != rank_Ab:
        print("Система несовместна (ранги не совпадают)")
    else:
        print("Система совместна")


------------ существование полинома 2 степени для t_stab[0] ------------



Подсчет времени стабилизации: 100%|██████████| 5000/5000 [00:37<00:00, 132.23it/s]


Размерность исходных признаков: 30
Размерность полиномиальных признаков: 495
Ранг матрицы A: 306
Ранг расширенной матрицы [A|B]: 307
Система несовместна (ранги не совпадают)

------------ существование полинома 2 степени для t_stab[1] ------------



Подсчет времени стабилизации: 100%|██████████| 5000/5000 [00:37<00:00, 133.73it/s]


Размерность исходных признаков: 30
Размерность полиномиальных признаков: 495
Ранг матрицы A: 306
Ранг расширенной матрицы [A|B]: 307
Система несовместна (ранги не совпадают)

------------ существование полинома 2 степени для t_stab[2] ------------



Подсчет времени стабилизации: 100%|██████████| 5000/5000 [00:37<00:00, 133.94it/s]


Размерность исходных признаков: 30
Размерность полиномиальных признаков: 495
Ранг матрицы A: 306
Ранг расширенной матрицы [A|B]: 306
Система совместна

------------- существование полинома 2 степени для t_total --------------



Подсчет времени стабилизации: 100%|██████████| 5000/5000 [00:37<00:00, 133.21it/s]


Размерность исходных признаков: 30
Размерность полиномиальных признаков: 495
Ранг матрицы A: 306
Ранг расширенной матрицы [A|B]: 307
Система несовместна (ранги не совпадают)
